<a href="https://colab.research.google.com/github/dhingratul/generative-ai-with-llm/blob/main/vector-db/chroma_palm_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# VectorDB with Chroma

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://developers.generativeai.google/examples/vectordb_with_chroma"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/examples/vectordb_with_chroma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/examples/vectordb_with_chroma.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


## Overview

This tutorial demonstrates how to use the PaLM API to create a vector database and retrieve answers to questions from the database. Moreover, you will use [ChromaDB](https://docs.trychroma.com/){:.external}, an open-source Python tool that creates embedding databases. ChromaDB allows you to:

* Store embeddings as well as their metadata
* Embed documents and queries
* Search through the database of embeddings

In this tutorial, you'll use embeddings to retrieve an answer from a database of vectors created with ChromaDB.

## Setup


First, download and install ChromaDB and the PaLM API Python library.

In [1]:
!pip install -q chromadb
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
import google.generativeai as palm

import pandas as pd

import chromadb
from chromadb.api.types import Documents, Embeddings

### Grab an API Key

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [5]:
palm.configure(api_key='PALM_KEY')

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the PaLM API is only available in the US.

In [6]:
models = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods]
model = models[0]
model

Model(name='models/embedding-gecko-001', base_model_id='', version='001', display_name='Embedding Gecko', description='Obtain a distributed representation of a text.', input_token_limit=1024, output_token_limit=1, supported_generation_methods=['embedText'], temperature=None, top_p=None, top_k=None)

### Data

Here is a small set of documents you will use to create an embedding database:

In [7]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

## Creating the embedding database with ChromaDB

You will create a [custom function](https://docs.trychroma.com/embeddings#custom-embedding-functions){:.external} for performing embedding using the PaLM API. By inputting a set of documents into this custom function, you will receive vectors, or embeddings of the documents.

In [8]:
def embed_function(texts: Documents) -> Embeddings:
  # Embed the documents using any supported method
  return  [palm.generate_embeddings(model=model, text=text)['embedding']
           for text in texts]

Now you will create the vector database. In the `create_chroma_db` function, you will instantiate a [Chroma client](https://docs.trychroma.com/getting-started){:.external}. From there, you will create a collection, which is where you store your embeddings, documents, and any metadata. Note that the embedding function from above is passed as an argument to the `create_collection`.

Next, you use the `add` method to add the documents to the collection.

In [9]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=embed_function)
  for i,d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [10]:
# Set up the DB
db = create_chroma_db([DOCUMENT1, DOCUMENT2, DOCUMENT3], "googlecardb")

Confirm that the data was inserted by looking at the database:

In [11]:
pd.DataFrame(db.peek(1))

,ids,embeddings,metadatas,documents
0,0,"[-0.01512377243489027, 0.05395153537392616, 0....",None,Operating the Climate Control System Your Goo...


## Getting the relevant document

`db` is a Chroma collection object. You can call `query` on it to perform a nearest neighbors search to find similar embeddings or documents.


In [12]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [13]:
# Perform embedding search
passage = get_relevant_passage("touch screen features", db)
passage

'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'

Now that you have found the relevant passage in your set of documents, you can use it make a prompt to pass into the PaLM API.

In [14]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

The `answer` function will generate a response based on the query you have passed in. It retrieves the relevant document, and from there calls the PaLM text generation API to generate a response to the query.

In [15]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
text_model = text_models[0]
text_model

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText'], temperature=0.7, top_p=0.95, top_k=40)

In [16]:
def answer(model, query, db, temperature=0.5):
  passage = get_relevant_passage(query, db)
  prompt = make_prompt(query, passage)
  answer = palm.generate_text(prompt=prompt, model=model, candidate_count=3, temperature=temperature, max_output_tokens=1000)
  return answer.candidates[0]['output']

The temperature controls the randomness of the output. The larger the value, the more random the generated text will be.

In [17]:
temperature = 0.65
query = "How do you shift gears in the Google car?"
answer(text_model, query, db, temperature)

'To shift gears in the Google car, simply move the shift lever to the desired position. The shift lever is located on the center console and has 6 positions: Park, Reverse, Neutral, Drive, Low, and Sport.'

## Next steps

You've now created your own embeddings database with ChromaDB and PaLM APIs! Try using your own data to create your own database as well. To learn more about how you can use the embeddings, check out the examples available. To learn how to use other services in the PaLM API, visit the various quickstart guides:

* [Chat quickstart](../tutorials/chat_quickstart.ipynb)

* [Text generation quickstart](../tutorials/text_quickstart.ipynb)